github link: https://github.com/ShlomiFridman/PhoenixProject2025

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install requests beautifulsoup4
!pip install requests beautifulsoup4 nltk
!pip install firebase

In [ ]:
import requests
import time
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from urllib.robotparser import RobotFileParser
from nltk.stem import PorterStemmer
import re
from firebase import firebase

Utils functions

In [ ]:
def read_txtfile(fileName):
    file_path = '/content/drive/My Drive/' + fileName
    with open(file_path, 'r') as file:
        allText = ""
        for line in file:
          allText += line
        wordsList =  allText.split()
        wordsSet =  set(wordsList)
    return wordsSet

#TODO add read image file

def index_words(soup):
    index_res = {}
    words = re.findall(r'\w+', soup.get_text())

    for word in words:
        word = word.lower()

        if word in index_res:
            index_res[word] += 1
        else:
            index_res[word] = 1

    return index_res

def remove_stop_words(p_index):
    stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at', 'to'}
    # stop_words = read_txtfile("stopwords_en.txt")

    for stop_word in stop_words:
        if stop_word in p_index:
            del p_index[stop_word]

    return p_index

def apply_stemming(p_index):
    stemmer = PorterStemmer()
    stemmed_index = {}

    for word, count in p_index.items():
        stemmed_word = stemmer.stem(word)

        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count

    return stemmed_index

Firebase service

In [ ]:
class FirebaseService:

  def __init__(self, db_url = 'https://phoenixhw2-default-rtdb.europe-west1.firebasedatabase.app/'):
    self.FBconn = firebase.FirebaseApplication(db_url,None)

  def get_index_from_DB(self):
    return self.FBconn.get('/Index/', None)

  def update_index_in_db(self, index_p):
    for k,v in index_p.items():
      self.FBconn.put("/Index/", k, v)
      print(f"update made for index={k} ({v['term']})")
    print("Updated given index in DB")

index service for maintain

In [ ]:
class IndexService:

  def __init__(self, index, firebaseService):
    self.firebaseService = firebaseService
    self.rev_index = {}
    self.urls_index = {}
    stemmer = PorterStemmer()
    for w in index:
      self.rev_index[stemmer.stem(w)] = {"term":w, "DocIDs": [], "DocIDs_cntrs": []}

  def process_soup(self, url, soup):
    if not soup:
      print("empty soup")
      return
    u_index = index_words(soup)
    u_index = remove_stop_words(u_index)
    u_index = apply_stemming(u_index)
    self.urls_index[url] = u_index

    for ind, ind_val in self.rev_index.items():
      if ind not in u_index:
        continue
      elif url in ind_val['DocIDs']:
        continue
      ind_val['DocIDs'].append(url)
      ind_val['DocIDs_cntrs'].append(u_index[ind])

    return u_index

  def get_reverse_index(self):
    return self.rev_index

  def get_url_index(self, url):
    return self.urls_index.get(url,{})

  def set_rev_index(self, newRevIndex):
    self.rev_index = newRevIndex
    self.urls_index = {}

    for ind, vals in self.rev_index.items():
      urls = vals['DocIDs']
      cntrs = vals['DocIDs_cntrs']
      for j in range(len(vals['DocIDs'])):
        if urls[j] not in self.urls_index:
          self.urls_index[urls[j]] = {}
        self.urls_index[urls[j]][ind] = cntrs[j]
    print("index updated")

  def add_new_word(self):
    # loop through url_index and build the word_dict for rev_index
    for urls in self.urls_index.keys():
        process_soup()

  def remove_word(self):
    # remove word from rev_index
    pass

  def add_new_url(self, url):
    # crawl the new url, get it's soup
    # build it's index, and add it to self.url_index
    # and update rev_index
    pass

  def remove_url(self, url):
    # update rev_index, every word that have the url remove it and it's cntr
    # remove the url from self.url_index
    pass

  def save_in_db(self):
    return firebaseService.save_rev_index(self.rev_index)
    # call firebaseService.save_rev_index(self.rev_index)

  def load_from_db(self):
    return self.set_rev_index(firebaseService.get_rev_index())
    #pass

  def index_toString(self):
    str = ''
    for ind,vals in self.rev_index.items():
      str += f"Index {ind}"
      str += f"\tTerm={vals['term']}"
      str += "\tDocIDs="
      for j in range(len(vals['DocIDs'])):
        str += f"\t\tURL No.{j}: {vals['DocIDs'][j]} - {vals['DocIDs_cntrs'][j]} times"
    return str


Crawling service

In [ ]:
class CrawlerService:

  def __init__(self, indexService):
    self.indexService = indexService
    self.crawled_urls = set()
    self.crawled_count = 0
    self.robot = None

    # Function to crawl a website and fetch n pages
  def crawl_website(self, base_url, max_pages):
      rp = self.__check_robot(base_url)  # Check the robots.txt file
      urls_to_crawl = [base_url]  # Initialize the queue with the base URL
      current_crawled_urls = set()
      ignore_urls = ["form", "mp3", "mp4", "downloads"]

      while urls_to_crawl and len(current_crawled_urls) < max_pages:
          current_url = urls_to_crawl.pop(0)

          if current_url in self.crawled_urls:
              continue  # Skip already crawled URLs

          print(f"Crawling {self.crawled_count+1}: {current_url}")
          page_content = self.__fetch_page_crawler(current_url, rp)

          if page_content:
              self.crawled_urls.add(current_url)
              current_crawled_urls.add(current_url)
              self.crawled_count += 1

              # Extract and queue new links to crawl
              new_links = self.__extract_links(page_content, base_url)
              for link in new_links:
                  cond1 = any(bad_url in link for bad_url in ignore_urls)
                  cond2 = any(u for u in self.crawled_urls if (f"{u}#" in link))
                  cond = cond1 or cond2
                  if link not in self.crawled_urls and link not in urls_to_crawl and not cond:
                      urls_to_crawl.append(link)

              soup = BeautifulSoup(page_content, 'html.parser')
              self.indexService.process_soup(current_url, soup)
              # Delay between requests to avoid overwhelming the server
              time.sleep(2)  # Sleep for 2 seconds between requests (politeness)

      print(f"\nCrawled {len(current_crawled_urls)} pages.")
      return current_crawled_urls

  def get_crawled_urls(self):
    return self.crawled_urls

  # Function to fetch and parse the robots.txt file to check permissions
  def __check_robot(self, url):
      robot_url = urljoin(url, '/robots.txt')
      rp = RobotFileParser()
      rp.set_url(robot_url)
      # Fetch and parse robots.txt file
      rp.read()
      if rp:
        self.robot = rp
      return rp if rp else self.robot

  # Function to fetch and parse a page
  def __fetch_page_crawler(self, url, rp):
      # Check if the URL is allowed to be crawled according to robots.txt
      if not rp.can_fetch('*', url):  # '*' means all user agents
          print(f"Blocked by robots.txt: {url}")
          return None

      try:
          response = requests.get(url, timeout=5)
          response.raise_for_status()  # Will raise an exception for 4xx or 5xx responses
          return response.text
      except requests.exceptions.RequestException as e:
          print(f"Error fetching {url}: {e}")
          return None

  # Function to extract internal links from a page
  def __extract_links(self, page_content, base_url):
      soup = BeautifulSoup(page_content, 'html.parser')
      links = set()

      # Find all anchor tags and extract the href attribute
      # TODO need to add all links that are in tags with property cta-type="local
      for anchor in soup.find_all('a', href=True):
          href = anchor['href']

          # Resolve relative URLs to absolute URLs
          full_url = urljoin(base_url, href)

          # Only add links that are within the same domain (ibm.com)
          if urlparse(full_url).netloc == urlparse(base_url).netloc:
              links.add(full_url)
      for tag in soup.find_all(attrs={'cta-type': 'local'}, href=True):
        href = tag['href']
        # Resolve relative URLs to absolute URLs
        full_url = urljoin(base_url, href)

        # Only add links that are within the same domain
        if urlparse(full_url).netloc == urlparse(base_url).netloc:
            links.add(full_url)

      return links

Query Service

In [ ]:
class QueryService:

  # enable boolean search

  def __init__(self, indexService):
    self.indexService = indexService
    # self.query_history_results = {}   # query => urls
    self.query_history = []   # query => urls

  def query(self, query):
    res=self.__query_process(query)
    self.query_history.insert(0, query)
    return res

  def get_history(self):
    return self.query_history

  def __query_process(self, query):
    if "AND" in query:
      ind=query.find("AND")
      return self.__andResults(self.__query_process(query[:ind]), self.__query_process(query[ind+3:]))
    elif "OR" in query:
      ind=query.find("OR")
      return self.__orResults(self.__query_process(query[:ind]), self.__query_process(query[ind+2:]))
    return self.__queryByString(query)

  def __queryByString(self, query):
    url_res_set = set()
    query_words = set(re.findall(r'\w+', query.lower()))
    stemmer = PorterStemmer()
    stemmed_query = set()
    rev_index = self.indexService.get_reverse_index()
    for word in query_words:
      stemmed_word = stemmer.stem(word)
      stemmed_query.add(stemmed_word)
      #add url to dict
      if stemmed_word in rev_index:
        url_res_set.update(rev_index[stemmed_word]["DocIDs"])

    ranked_url_res = []
    for url_val in url_res_set:
      ranked_url_res.append({'url':url_val, 'rank':self.rank_url(url_val, stemmed_query)})
    ranked_url_res = sorted(ranked_url_res, key=lambda item: item['rank'], reverse=True)
    # add result to history
    # self.query_history[query] = ranked_url_res
    # self.query_history.append({'query':query, 'results':ranked_url_res})
    # print(type(ranked_url_res))
    return ranked_url_res

  def __andResults(self, lst1, lst2):
    res_dict = {item['url']: item['rank'] for item in lst2}
    intersection = []
    for item in lst1:
        url = item['url']
        if url in res_dict:
            new_rank = (item['rank'] + res_dict[url])/2
            intersection.append({'url': url, 'rank': new_rank})

    # Sort the result by rank in descending order
    return sorted(intersection, key=lambda item: item['rank'], reverse=True)

  def __orResults(self, lst1, lst2):
    combined_dict = {}

    for item in lst1:
        url = item['url']
        rank = item['rank']
        combined_dict[url] = max(combined_dict.get(url, float('-inf')), rank)
    for item in lst2:
        url = item['url']
        rank = item['rank']
        combined_dict[url] = max(combined_dict.get(url, float('-inf')), rank)

    return sorted(
        [{'url': url, 'rank': rank} for url, rank in combined_dict.items()],
        key=lambda item: item['rank'],
        reverse=True)

  def rank_url(self, url, query_words):
    rank = 1
    # resultService = ResultService()
    # rank based on lab6
    url_index = self.indexService.get_url_index(url)
    for word in query_words:
      if word in url_index:
        rank = rank*1/url_index[word]
    rank = 1-rank
    # print(rank)
    return rank

  # def get_history(self):
  #   return self.query_history



Result Service

In [ ]:
# # Lab 7
# # result_service.py
# class HistoryService:
#     def __init__(self):
#         self.history = {}

#     def add_to_history(self, query_id, ):
#         """Format search results for display"""
#         try:
#             query = self.query_service.queries.get(query_id)
#             if not query:
#                 return {'error': 'Query not found'}

#             formatted_results = []
#             for doc_id in query['results']:
#                 doc = self.index_service.get_document(doc_id)
#                 if doc:
#                     formatted_results.append({
#                         'doc_id': doc_id,
#                         'title': doc['title'],
#                         'snippet': doc['content'][:100] + '...'
#                     })

#             result_id = str(len(self.results) + 1)
#             result = {
#                 'id': result_id,
#                 'query_id': query_id,
#                 'formatted_results': formatted_results,
#                 'count': len(formatted_results)
#             }
#             self.results[result_id] = result
#             return result

#         except Exception as e:
#             return {'error': str(e)}

#     def get_history(self):
#       return self.query_history

The index we defined

In [ ]:
init_index = [
    'SAAS',
    'PAAS',
    'IAAS',
    'FAAS',
    'Private',
    'Public',
    'Hybrid',
    'Service',
    'Platform',
    'Infrastructure',
    'Study',
    'Case',
    'Chatbot',
    'Engine',
    'Cloud',
    'Monitor',
    'Data',
    'Mainframe',
    'Performance',
    'Security',
    'SLA',
    'KPI',
    'SOA',
    'Information',
    'Kafka',
    'SQL',
    'Technology',
    'Database',
    'Docker',
    'Kubernetes',
    'RabbitMQ',
    'IBM',
    'Google',
    'Amazon',
    'AI',
    'Artificial',
    'Intelligence',
]

In [ ]:
firebaseService = FirebaseService()

In [ ]:
indexService = IndexService(init_index, firebaseService)


In [ ]:
crawlerService = CrawlerService(indexService)

In [ ]:
indexService.set_rev_index(firebaseService.get_index_from_DB())
# print(indexService.index_toString())

index updated


In [ ]:
queryService = QueryService(indexService)
query1 = queryService.query("PAAS")
# query2 = queryService.query("SAAS")
query3 = queryService.query("SAAS OR PAAS")
# query4 = queryService.query("SAAS AND PAAS")
print(query1)
# print(query2)
print(query3)
# print(query4)
print(queryService.get_history())

[{'url': 'https://www.ibm.com/topics/paas', 'rank': 0.9824561403508771}, {'url': 'https://www.ibm.com/topics/serverless', 'rank': 0.8888888888888888}, {'url': 'https://www.ibm.com/topics/cloud-computing', 'rank': 0.8571428571428572}, {'url': 'https://www.ibm.com/topics/middleware', 'rank': 0.8333333333333334}, {'url': 'https://www.ibm.com/topics/saas', 'rank': 0.75}, {'url': 'https://www.ibm.com/topics', 'rank': 0.5}, {'url': 'https://www.ibm.com/topics/cloud-security', 'rank': 0.5}, {'url': 'https://www.ibm.com/topics/it-management', 'rank': 0.5}, {'url': 'https://www.ibm.com/topics/virtualization', 'rank': 0.5}, {'url': 'https://www.ibm.com/topics/infrastructure', 'rank': 0.5}, {'url': 'https://www.ibm.com/topics/vpc', 'rank': 0.0}, {'url': 'https://www.ibm.com/topics/block-storage', 'rank': 0.0}, {'url': 'https://www.ibm.com/topics/etcd', 'rank': 0.0}]
[{'url': 'https://www.ibm.com/topics/paas', 'rank': 0.9824561403508771}, {'url': 'https://www.ibm.com/topics/saas', 'rank': 0.981818

In [ ]:
indexService.urls_index["https://www.ibm.com/us-en"]
#indexService.add_new_word()

{'ai': 28,
 'cloud': 3,
 'data': 4,
 'databas': 2,
 'hybrid': 2,
 'ibm': 20,
 'infrastructur': 1,
 'perform': 1,
 'secur': 3,
 'studi': 2,
 'technolog': 6}

In [ ]:
indexService.get_reverse_index()

{'ai': {'DocIDs': ['https://www.ibm.com/us-en',
   'https://www.ibm.com/impact/ai-ethics?lnk=bus',
   'https://www.ibm.com/servers?lnk=ProdC',
   'https://www.ibm.com/thought-leadership/institute-business-value/report/ceo-generative-ai?lnk=bus',
   'https://www.ibm.com/think/reports/ai-in-action?lnk=bus',
   'https://www.ibm.com/think/insights/ai-ethics-and-governance-in-2025?lnk=hpUSls2',
   'https://www.ibm.com/products/watsonx-code-assistant?lnk=dev',
   'https://www.ibm.com/about?lnk=inside',
   'https://www.ibm.com/automation?lnk=ProdC',
   'https://www.ibm.com/history?lnk=inside',
   'https://www.ibm.com/analytics?lnk=ProdC',
   'https://www.ibm.com/artificial-intelligence?lnk=ProdC',
   'https://www.ibm.com/new/announcements/scale-ai-with-ibm-s-forward-thinking-data-strategy?lnk=hpUSrc3',
   'https://www.ibm.com/hybrid-cloud?lnk=ProdC',
   'https://www.ibm.com/consulting?lnk=ProdC',
   'https://www.ibm.com/security?lnk=ProdC',
   'https://www.ibm.com/granite?lnk=ProdC',
   'http

In [ ]:
#משימה ששלומי נתן לישראל 31.12.24
#בוצעה
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
from collections import defaultdict


class GraphService:
    def __init__(self, rev_index):
        """
        Initializes the GraphService with the reverse index data.
        """
        self.rev_index = rev_index
        self.index_df_coalition_heatmap = None
        self.index_df_urls_heatmap = None
        self.index_df_bar = None
        self.__buildDFs(rev_index)

    def __buildDFs(self, rev_index):
        """
        Builds the dataframes for heatmap and bar chart from the reverse index data.
        """
        self.index_df_coalition_heatmap = self.__get_shared_docs_dataframe()

        # Collect data for the heatmap
        webpage_data = defaultdict(lambda: {"words": [], "occurrences": []})
        for key, value in rev_index.items():
            doc_ids = value.get("DocIDs", [])
            doc_ids_cntrs = value.get("DocIDs_cntrs", [])
            term = value.get("term", "")

            for doc_id, count in zip(doc_ids, doc_ids_cntrs):
                webpage_data[doc_id]["words"].append(term)
                webpage_data[doc_id]["occurrences"].append(count)

        # Create DataFrame for heatmap
        webpage_dataframes = {}
        for page, content in webpage_data.items():
            webpage_df = pd.DataFrame({
                "words": content["words"],
                "occurrences": content["occurrences"]
            })
            webpage_dataframes[page] = webpage_df

        self.index_df_urls_heatmap = pd.DataFrame({
            page: {row["words"]: row["occurrences"] for _, row in df.iterrows()}
            for page, df in webpage_dataframes.items()
        }).fillna(0)

        # Create DataFrame for bar chart
        word_totals = self.index_df_urls_heatmap.sum(axis=1).reset_index()
        word_totals.columns = ["word", "occurrences"]
        self.index_df_bar = word_totals

    def get_heatmap(self):
        """
        Returns the heatmap graph wrapped in a widget output.
        """
        heatmap_output = widgets.Output()
        with heatmap_output:
            print("Heatmap: Number of Shared URLs Between Indexes")
            plt.figure(figsize=(20,10))
            sns.heatmap(
                self.index_df_coalition_heatmap,
                annot=True,
                cmap="YlGnBu",
                linewidths=0.5,
                fmt="g",
                cbar_kws={'label': 'Occurrences'}
            )
            plt.title("Heatmap: Shared URLs Between Indexes", fontsize=14)
            plt.xlabel("Index 1", fontsize=12)
            plt.ylabel("Index 2", fontsize=12)
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()
        return heatmap_output




    def get_barChart(self):
        """
        Returns the bar chart graph wrapped in a widget output, with bars sorted in descending order.
        """
        bar_chart_output = widgets.Output()
        with bar_chart_output:
            print("Bar Chart: Word Occurrences Across Webpages")

            # Sort the DataFrame in descending order of occurrences
            sorted_df = self.index_df_bar.sort_values(by="occurrences", ascending=False)

            plt.figure(figsize=(14, 8))
            sns.barplot(data=sorted_df, x="word", y="occurrences", palette="viridis")
            plt.title("Bar Chart: Word Occurrences Across Webpages (Sorted)", fontsize=16)
            plt.xlabel("Words", fontsize=14)
            plt.ylabel("Occurrences", fontsize=14)
            plt.xticks(rotation=45, fontsize=10)
            plt.tight_layout()
            plt.show()
        return bar_chart_output





    def __get_shared_docs_dataframe(self):
        """
        Creates a dataframe of shared documents between terms.
        """
        reverse_index = self.rev_index

        count_shared_docs = lambda docs1, docs2: len(set(docs1) & set(docs2))

        indices = list(reverse_index.keys())
        data = {}

        for index1 in indices:
            data[reverse_index[index1]["term"]] = {}
            for index2 in indices:
                shared_docs_count = count_shared_docs(
                    reverse_index[index1]["DocIDs"], reverse_index[index2]["DocIDs"]
                )
                data[reverse_index[index1]["term"]][reverse_index[index2]["term"]] = shared_docs_count

        df = pd.DataFrame(data)

        for i in range(len(df)):
            df.iloc[i, i] = 0  # Set diagonal to 0
            for j in range(i + 1, len(df)):
                df.iloc[i, j] = 0  # Set values above the diagonal to 0

        return df


In [ ]:
#משימה ששלומי נתן לישראל 31.12.24
#בוצעה
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

class SearchEngineUI:
    def __init__(self, indexService):
        """
        Initializes the SearchEngineUI with the indexService.
        """
        self.indexService = indexService
        self.query_input = widgets.Text(
            placeholder="Enter your search query here...",
            description="Query:",
            layout=widgets.Layout(width='70%')
        )
        self.search_button = widgets.Button(
            description="Search",
            button_style="primary",
            tooltip="Click to search",
            icon="search"
        )
        self.results_output = widgets.Output()

        # Attach event handler
        self.search_button.on_click(self.perform_search)

        # Layout the GUI
        self.gui = widgets.VBox([
            widgets.HBox([self.query_input, self.search_button]),
            self.results_output
        ])

    def display(self):
        """
        Displays the GUI for the SearchEngineUI.
        """
        display(self.gui)

    def query(self, query_str):
        """
        Executes a query and returns the results.
        """
        try:
            results = self.indexService.query(query_str)  # Assuming indexService.query() returns a list of JSON results
            # Sort results by rank in descending order
            sorted_results = sorted(results, key=lambda x: x['rank'], reverse=True)
            return sorted_results
        except Exception as e:
            print(f"An error occurred during the query: {e}")
            return []

    def perform_search(self, b):
        """
        Executes the search and displays results when the search button is clicked.
        """
        query = self.query_input.value.strip()
        if not query:
            with self.results_output:
                self.results_output.clear_output()
                print("Please enter a search query.")
            return

        # Get the results for the query
        results = self.query(query)

        # Display results
        with self.results_output:
            self.results_output.clear_output()
            if not results:
                print("No results found for your query.")
            else:
                # Convert results to a DataFrame and keep only the 'url' column
                results_df = pd.DataFrame(results)
                if 'url' in results_df.columns:
                    # Add numbering and clickable links
                    results_df = results_df[['url']]
                    results_df['url'] = results_df['url'].apply(
                        lambda x: f'<a href="{x}" target="_blank">{x}</a>'
                    )

                    # Create HTML with alternating white and gray lines
                    numbered_results = [
                        f'<div style="background-color: {"#f9f9f9" if i % 2 == 0 else "#eaeaea"}; padding: 8px;">'
                        f"{i + 1}. {url}"
                        f"</div>"
                        for i, url in enumerate(results_df['url'])
                    ]

                # Display the results with alternating colors
                html = '<div style="text-align: left; font-family: Arial; font-size: 14px;">' + ''.join(
                    numbered_results
                ) + '</div>'
                display(HTML(html))



In [ ]:
#משימה ששלומי נתן לישראל 31.12.24
#בוצעה
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

def display_tabs(search_ui, history_service, heatmap_output, bar_chart_output, edit_index_output):
    """
    Creates and displays the tabs containing the search engine, history, and graphs.
    """
    # Create a container for the search engine UI in the first tab
    query_service_output = widgets.Output()
    with query_service_output:
        search_ui.display()

    # Create the tabs
    tabs = widgets.Tab(children=(
        query_service_output,
        history_service,
        heatmap_output,
        bar_chart_output,
        edit_index_output
        ))
    tabs.set_title(0, "Query Service")
    tabs.set_title(1, "History Service")
    tabs.set_title(2, "Heatmap")
    tabs.set_title(3, "Bar Chart")
    tabs.set_title(4, "Edit Index Menu")
    display(tabs)

# Assuming GraphService and indexService are already defined
graphService = GraphService(indexService.get_reverse_index())

# Generate the outputs for each graph
heatmap_output = graphService.get_heatmap()
bar_chart_output = graphService.get_barChart()

# Define the history service output
history_output = widgets.Output()
with history_output:
    print("History Service: Currently empty.")

edit_index_output = widgets.Output()
with edit_index_output:
    print("Edit Index Menu: Currently empty.")

# Assuming SearchEngineUI and queryService are already defined
search_ui = SearchEngineUI(queryService)

# Display the tabs with the search engine and other services
display_tabs(search_ui, history_output, heatmap_output, bar_chart_output, edit_index_output)


In [ ]:
#מפה ומטה אסור להריץ ללא אישור של שלומי כי יש פה משהו שמאפס את המסד נתונים

# q = "adsadasdasdas"
# q_history = historyService.get_history_of(q)
# if q_history:
#   pass  # display this as result
# else:
#   q_res = queryService.query(q)
#   historyService.add_to_history(q_res)
#   pass  # display this as result


In [ ]:
# # main.py
# def main():
#     # Initialize services
#     indexService = IndexService(init_index)
#     crawlerService = CrawlerService(indexService)
#     # resultService = ResultService(indexService, queryService)

# #if __name__ == "__main__":
# #  main()

In [ ]:

# # To limit the number of pages to crawl
# MAX_PAGES = 10
# crawlerService.crawl_website('https://www.ibm.com/us-en', MAX_PAGES)
# crawlerService.crawl_website('https://www.ibm.com/topics', MAX_PAGES)

In [ ]:
# firebaseService.update_index_in_db(indexService.get_reverse_index())

TODO 3 tabs: search results, graph that shows the rank of each page, graph for the website cover of the query keywords

TODO add the group logo from drive

TODO enable shering, make the link public

TODO edit_index: print_index, add_new_word, remove_from_index, add_url, remove_url, get_index_from_db, save_index_in_db, exit menu